In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
pip install plotly

In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

import seaborn as sns
from collections import OrderedDict, Counter
import os
import networkx as nx
import matplotlib.pyplot as plt
from nltk import word_tokenize
import string
from datetime import datetime
import plotly.graph_objs as go

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Longitudinal palavras

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/tweets_total.csv', encoding = 'utf-8')
df = df.dropna()
#df = df[(df['date'] > '2013-05-01 00:00:00') & (df['date'] < '2013-07-31 09:00:00')]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (0,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
def word_transform(words):
    words = words.split()
    total = [x.lower() for x in words]
    manual_list_stop_words = ['só','disse','ser','sobre','ainda','se','além', 'outros','porque','assim','ter', 'access_time', 'more_horiz',
                             'out','jun','publicado','diz','exame','patrocinado','ago','vip','abr','http','vai','HTTP'
                             'jul','jan','abr','set','nov','dez','conteúdo','mar','maio','estadão','marketing','pictwittercom','dia','mil','pra',
                             'fev','paulo', 'the','após', 'redação', 'foto', 'feira', 'segundo', 'nesta', 'sexta','contra','governo','dilma'
            'onde','pode','sábado','segunda','três', 'fazer','todos','parte','local', 'região', 'disse','quarta','nova','novo',
            'via','veja','neste','nm','aponta','mostra','rt','vejaabrilcombr','folha','vai','desde','do','multimidia','vejacom','blog', '-', '@','"','...','dilma']
    stop_words = stopwords.words('portuguese')
    for i in manual_list_stop_words:
        stop_words.append(i)
    
    total = [x for x in total if x not in stop_words]
    total = [re.sub(r'[,\.!?()]', '', x) for x in total]
    total = [x for x in total if not re.search(r"\d", x)]
    
    long_string = (" ").join(total)
    return long_string

In [ ]:
df['tweet'] = df['tweet'].apply(word_transform)
#df['tweet'] = df['tweet'].apply(lambda x: x.split())

KeyboardInterrupt: ignored

In [ ]:
df_new = df[['date','tweet']]
df_new.columns = ['Data','Texto']

In [ ]:
df_new['Data'] = df_new['Data'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
df_new['Data'] = df_new['Data'].apply(lambda x: x.to_period('M'))
df_new['Data'] = df_new['Data'].apply(lambda x: x.to_timestamp())

In [ ]:
df_new = df_new.groupby(['Data'])['Texto'].apply(' '.join).reset_index()

In [ ]:
def contar_palavra_corrupcao(row):
    soma = 0
    for i in row.split():
        if i == 'corrupção' or i == 'corrupcao':
            soma+=1
    return soma

def contar_protesto(row):
    soma = 0
    for i in row.split():
        if i == 'protesto':
            soma+=1
    return soma  

def contar_lavajato(row):
    soma = 0
    for i in row.split():
        if i == 'lavajato' or i =='lava jato' or i == 'lava-jato':
            soma+=1
    return soma 

def aprovacao(row):
    soma = 0
    for i in row.split():
        if i == 'aprovação':
            soma+=1
    return soma 

def pesquisa(row):
    soma = 0
    for i in row.split():
        if i == 'pesquisa':
            soma+=1
    return soma 

def ibope(row):
    soma = 0
    for i in row.split():
        if i == 'ibope':
            soma+=1
    return soma 

def dilma(row):
    soma = 0
    for i in row.split():
        if i == 'dilma':
            soma+=1
    return soma 

def lula(row):
    soma = 0
    for i in row.split():
        if i == 'lula':
            soma+=1
    return soma 

def datafolha(row):
    soma = 0
    for i in row.split():
        if i == 'datafolha':
            soma+=1
    return soma 

def brasil(row):
    soma = 0
    for i in row.split():
        if i == 'brasil':
            soma+=1
    return soma 



In [ ]:
df_new['coorupcao_freq'] = df_new['Texto'].apply(contar_palavra_corrupcao)
df_new['protesto_freq'] = df_new['Texto'].apply(contar_protesto)
df_new['lavajato_freq'] = df_new['Texto'].apply(contar_lavajato)
df_new['aprovacao'] = df_new['Texto'].apply(aprovacao)
df_new['pesquisa'] = df_new['Texto'].apply(pesquisa)
df_new['ibope'] = df_new['Texto'].apply(ibope)
df_new['dilma'] = df_new['Texto'].apply(dilma)
df_new['lula'] = df_new['Texto'].apply(lula)
df_new['datafolha'] = df_new['Texto'].apply(datafolha)
df_new['brasil'] = df_new['Texto'].apply(brasil)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['coorupcao_freq'], name = 'Corrupção'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['protesto_freq'], name = 'Protesto'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['aprovacao'], name = 'Aprovação'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['lavajato_freq'], name = 'Lava jato'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['pesquisa'], name = 'pesquisa'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['ibope'], name = 'ibope'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['dilma'], name = 'dilma'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['lula'], name = 'lula'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['datafolha'], name = 'datafolha'))
fig.add_trace(go.Scatter(x=df_new['Data'], y=df_new['brasil'], name = 'brasil'))
fig.update_xaxes(rangeslider_visible=True)
fig.show()

#### Longitudinal quantidade Tweets


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/tweets_total.csv', encoding = 'utf-8')
df = df.dropna()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (0,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
df['count'] = 1 
df_new = df[['date','count']]
df_new.columns = ['Data','count']

df_new['Data'] = df_new['Data'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
df_new['Data'] = df_new['Data'].apply(lambda x: x.to_period('M'))
df_new['Data'] = df_new['Data'].apply(lambda x: x.to_timestamp())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
df_long = df_new.groupby(['Data'])['count'].sum().reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_long['Data'], y=df_long['count'], name = 'Quantidade de Tweets'))
fig.update_xaxes(rangeslider_visible=True)
fig.show()

### Sentimento

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/tweets_total.csv', encoding = 'utf-8')
df = df.dropna()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (0,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
def word_transform(words):
    words = words.split()
    total = [x.lower() for x in words]
    manual_list_stop_words = ['só','disse','ser','sobre','ainda','se','além', 'outros','porque','assim','ter', 'access_time', 'more_horiz',
                             'out','jun','publicado','diz','exame','patrocinado','ago','vip','abr','http','vai','HTTP'
                             'jul','jan','abr','set','nov','dez','conteúdo','mar','maio','estadão','marketing','pictwittercom','dia','mil','pra',
                             'fev','paulo', 'the','após', 'redação', 'foto', 'feira', 'segundo', 'nesta', 'sexta','contra','governo','dilma'
            'onde','pode','sábado','segunda','três', 'fazer','todos','parte','local', 'região', 'disse','quarta','nova','novo',
            'via','veja','neste','nm','aponta','mostra','rt','vejaabrilcombr','folha','vai','desde','do','multimidia','vejacom','blog', '-', '@','"','...','dilma', 'faz']
    stop_words = stopwords.words('portuguese')
    for i in manual_list_stop_words:
        stop_words.append(i)
    
    total = [x for x in total if x not in stop_words]
    total = [re.sub(r'[,\.!?()]', '', x) for x in total]
    total = [x for x in total if not re.search(r"\d", x)]
    
    long_string = (" ").join(total)
    return long_string

In [ ]:
df['tweet'] = df['tweet'].apply(word_transform)

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: x.split())

In [ ]:
df_new = df[['date','tweet']]
df_new['count'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_pol = pd.read_csv('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/oplexicon_v3.0/lexico_v3.0.txt', skiprows=500, encoding = 'utf-8',usecols=[0,2])
df_pol.columns = ['palavra','polaridade']

lista_positivas = df_pol[df_pol['polaridade'] > 0]['palavra'].values.tolist()
lista_negativas = df_pol[df_pol['polaridade'] < 0]['palavra'].values.tolist()

In [ ]:
def count_positivo(x, lista_positivas):

  number_set = len(set(x).intersection(lista_positivas)) 
  return number_set

def count_negativo(x, lista_negativas):
  number_set = len(set(x).intersection(lista_negativas)) 
  return number_set


In [ ]:
%%time
df_new['positivo_qtd'] = df_new['tweet'].apply(lambda x: count_positivo(x,lista_positivas=lista_positivas))

CPU times: user 1min 11s, sys: 9.98 ms, total: 1min 11s
Wall time: 1min 11s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
%%time
df_new['negativo_qtd'] = df_new['tweet'].apply(lambda x: count_negativo(x,lista_negativas=lista_negativas))


CPU times: user 1min 55s, sys: 11.3 ms, total: 1min 55s
Wall time: 1min 55s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_new['positivo'] = df_new['positivo_qtd'] - df_new['negativo_qtd']
df_new['negativo'] = df_new['negativo_qtd'] - df_new['positivo_qtd'] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def new(x):
  if x > 0:
    return 1
  else:
    return 0

df_new['positivo'] = df_new['positivo'].apply(new)
df_new['negativo'] = df_new['negativo'].apply(new)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_new_pol = df_new[['date','count','positivo','negativo']]

df_new_pol['date'] = df_new_pol['date'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
df_new_pol['date'] = df_new_pol['date'].apply(lambda x: x.to_period('M'))
df_new_pol['date'] = df_new_pol['date'].apply(lambda x: x.to_timestamp())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
df_long = df_new.groupby(['date'])['count','positivo','negativo'].sum().reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
fig = go.Figure()
#fig.add_trace(go.Scatter(x=df_long['date'], y=df_long['count'], name = 'Quantidade de Tweets'))
fig.add_trace(go.Scatter(x=df_long['date'], y=df_long['positivo'], name = 'Tweets Positivos'))
fig.add_trace(go.Scatter(x=df_long['date'], y=df_long['negativo'], name = 'Tweets Negativos'))

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
df_long[(df_long['date'] > '2014-10-01') & (df_long['date'] < '2016-08-30')]['positivo'].sum()

50716

In [ ]:
df_long[(df_long['date'] > '2014-10-01') & (df_long['date'] < '2016-08-30')]['negativo'].sum()

60533

In [ ]:
INSERT INTO ri5_cemig_inad.action_types VALUES
(1,NULL,'AT_Inspection',NULL, NULL)
(3,	'Corte e Recorte',	'AT_Corte',NULL, NULL)
(4,	'E-mail Cobrança',	'AT_Email',NULL, NULL)
(5,	'SMS',	'AT_Sms',NULL, NULL)
(6,	'Carta cobrança',	'AT_CartaCob',NULL, NULL)
(7,	'Negativação',	'AT_Neg',NULL, NULL)
(8,	'Cobrança Judicial',	'AT_Jud',NULL, NULL)
(9,	'Plataforma On-line',	'AT_Online',NULL, NULL)
(10,	'Protesto',	'AT_Protesto',NULL, NULL)
(11,	'Campanha E-mail',	'AT_CampEmail',NULL, NULL)
(12,	'Campanha SMS',	'AT_CampSMS',NULL, NULL)
(13,	'Campanha Carta',	'AT_CampCarta',NULL, NULL)
(14,	'WhatsApp',	'AT_WhatsApp',NULL, NULL)
(15,	'URA',	'AT_URA',NULL, NULL)
(16,	'Reaviso fora da conta',	'AT_Reaviso',NULL, NULL)
(17,	'Cobrança Administrativa',	'AT_Adm',NULL, NULL)
(18,	'E-mail fatura a vencer',	'AT_EmailFat',NULL, NULL)
(19,	'SMS fatura a vencer',	'AT_SMSFat',NULL, NULL)
(20,	'WhatsApp fatura a vencer',	'AT_WppFat',NULL, NULL)